<a href="https://colab.research.google.com/github/karthikugn21/Sketch-to-image/blob/main/Sketch_to_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import torch
from diffusers import StableDiffusionXLPipeline, ControlNetModel, UniPCMultistepScheduler
from PIL import Image, ImageOps
import gradio as gr

def preprocess_sketch_pil(img: Image.Image, image_size: int = 1024) -> Image.Image:
    img = img.convert("L")
    img = ImageOps.invert(img)
    img = img.resize((image_size, image_size))
    img = img.convert("RGB")
    return img

def load_pipeline():
    controlnet = ControlNetModel.from_pretrained(
        "xinsir/controlnet-scribble-sdxl-1.0",
        torch_dtype=torch.float16
    )
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        controlnet=controlnet,
        torch_dtype=torch.float16,
        variant="fp16"
    )
    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
    pipe.to("cuda")
    return pipe

pipe = load_pipeline()

def infer(sketch: Image.Image, prompt: str) -> Image.Image:
    sketch = preprocess_sketch_pil(sketch)
    result = pipe(prompt=prompt, image=sketch, num_inference_steps=30)
    return result.images[0]

def main():
    sketch_input = gr.Image(label="Upload Sketch Image", type="pil")
    prompt_input = gr.Textbox(lines=1, placeholder="Enter a prompt...", label="Prompt")

    interface = gr.Interface(
        fn=infer,
        inputs=[sketch_input, prompt_input],
        outputs=gr.Image(type="pil", label="Generated Image"),
        title="Sketch to Image",
        description="Upload a sketch and enter a prompt. This app uses Stable Diffusion XL with ControlNet for high-fidelity image generation."
    )
    interface.launch()

if __name__ == "__main__":
    main()
